# Calculate normalised enrichment scores of Danaher gene sets

Script related to figure 3a, 3b, s3c 

### Aim:
- Calculate NES of Danaher Gene Sets

In [1]:
sessionInfo()

R version 3.5.0 (2018-04-23)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS High Sierra 10.13.6

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] compiler_3.5.0  IRdisplay_0.6.1 pbdZMQ_0.3-3    tools_3.5.0    
 [5] htmltools_0.3.6 base64enc_0.1-3 crayon_1.3.4    Rcpp_1.0.1     
 [9] uuid_0.1-2      IRkernel_0.8.14 jsonlite_1.6    digest_0.6.18  
[13] repr_0.17       evaluate_0.13  

## Load packages

In [1]:
library(GSVA)
library(BBmisc)
library(nortest)

Warning message:
“package ‘GSVA’ was built under R version 3.5.1”
Attaching package: ‘BBmisc’

The following object is masked from ‘package:base’:

    isFALSE



In [2]:
setwd("~/git_repos/HGSOC_TME_Heterogeneity/Scripts/3/")

## Load expression data

In [ ]:
mix <- "../../Data/1/TreatmentNaive_log2exp_loess_norm.txt"

In [3]:
data <- read.csv(mix,
                 sep='\t',
                 header=T,
                 row.names='Hugo_Symbol')

In [4]:
head(data)

,A01,A02,A03,A04,A05,A06,A07,A08,A09,A10,⋯,C06,C07,C08,C09,C10,C11,C12,D01,D02,D03
A1BG,4.124711,3.880633,4.864973,4.167882,4.317845,3.679057,4.092837,4.370571,4.252637,4.174694,⋯,3.804537,4.416981,4.169014,4.503313,4.146333,4.078398,4.011369,3.845128,3.600927,4.521857
A2BP1,3.960924,4.033715,3.721251,3.916073,4.338194,3.357396,3.872447,3.803388,3.484735,4.427405,⋯,4.991068,3.799913,4.117883,4.122134,4.221760,3.648326,3.675317,3.546925,3.788279,4.920841
A2M,12.603155,10.859838,12.612923,9.176999,10.548790,11.840776,7.200547,11.835658,11.535547,7.080118,⋯,9.061598,9.189459,8.844728,7.464440,6.726233,9.265911,10.740486,9.735582,10.781599,8.734304
A2ML1,3.945877,4.000621,3.767932,3.789729,3.543204,3.805520,4.336280,4.031757,4.166018,3.673146,⋯,4.231174,3.871651,3.287232,3.721093,3.723801,3.884499,3.694111,3.937682,3.600471,3.402861
A4GALT,3.597277,3.863757,4.051844,4.113169,3.975144,3.991293,5.439960,4.696177,4.171788,4.041633,⋯,4.183089,4.699729,4.019537,3.739865,4.591953,4.997860,4.099630,5.001239,4.714710,4.145119
A4GNT,3.714481,3.593169,3.241352,3.515850,3.364447,3.767865,3.768695,3.163977,3.423377,3.205154,⋯,3.896812,3.915653,3.351422,3.719759,3.641028,3.114521,3.560199,3.539738,3.760903,3.479266


In [5]:
data_mtx <- as.matrix(data)

## Load Danaher gene sets

In [6]:
Danaher_GeneSets <- read.csv('../../Data/3/DanaherImmuneCells_GeneSets.txt',
                             sep='\t',
                             header=T)

In [7]:
head(Danaher_GeneSets)

B.cells,CD45,CD8.T.cells,Cytotoxic.cells,DC,Exhausted.CD8,Macrophages,Mast.cells,Neutrophils,NK.CD56dim.cells,NK.cells,T.cells,Th1.cells,Treg
BLK,PTPRC,CD8A,CTSW,CCL13,CD244,CD163,MS4A2,CSF3R,IL21R,NCR1,CD3D,TBX21,FOXP3
CD19,,CD8B,GNLY,CD209,EOMES,CD68,TPSAB1,S100A12,KIR2DL3,XCL2,CD3E,,
MS4A1,,,GZMA,HSD11B1,LAG3,CD84,CPA3,CEACAM3,KIR3DL1,XCL1,CD3G,,
TNFRSF17,,,GZMB,,PTGER4,MS4A4A,HDC,FCAR,KIR3DL2,,CD6,,
FCRL2,,,GZMH,,,,TPSB2,FCGR3B,,,SH2D1A,,
KIAA0125,,,KLRB1,,,,,FPR1,,,TRAT1,,


In [8]:
Danaher_GeneSets <- convertRowsToList(t(Danaher_GeneSets))

## Run ssGSEA

In [9]:
dan_NES <- gsva(data_mtx,
                Danaher_GeneSets,
                method='ssgsea',
                min.sz=0,
                max.sz=1000,
                ssgsea.norm=T)

Estimating ssGSEA scores for 14 gene sets.
  |                                                                      |   0%Using parallel with 4 cores
  |======================================================================| 100%


In [10]:
dan_NES <- data.frame("Term"=rownames(dan_NES),
                      dan_NES)

rownames(dan_NES) <- NULL

In [11]:
head(dan_NES)

Term,A01,A02,A03,A04,A05,A06,A07,A08,A09,⋯,C06,C07,C08,C09,C10,C11,C12,D01,D02,D03
B.cells,-0.21810261,-0.25335475,-0.25329705,-0.27594330,-0.278792671,-0.26363070,-0.2885520,-0.31224478,-0.30345988,⋯,-0.26713176,-0.29081847,-0.29290030,-0.27643295,-0.2969665,-0.09979159,-0.18757817,-0.20150193,-0.18565465,-0.20666753
CD45,0.49598435,0.49069879,0.49069879,0.37386738,0.453562603,0.45651428,0.2852485,0.36007002,0.30584157,⋯,0.43461697,0.47298874,0.40372735,0.44477622,0.4188976,0.49090472,0.48156919,0.48706068,0.48500137,0.38608594
CD8.T.cells,-0.44253789,-0.33635556,-0.45810935,-0.42823210,-0.372922238,-0.48621331,-0.4367254,-0.45148510,-0.48466856,⋯,-0.40361036,-0.44692447,-0.49666450,-0.31303111,-0.3606616,-0.41661137,-0.35611691,-0.45677517,-0.33679621,-0.48221195
Cytotoxic.cells,0.05947407,0.05165459,0.03771431,-0.09735425,-0.027969728,-0.08358612,-0.1698657,-0.09678797,-0.18403653,⋯,-0.18732346,-0.14627329,-0.17240717,-0.13248797,-0.1633762,-0.03742641,-0.03825606,-0.05396364,-0.06621261,-0.13739278
DC,-0.10055686,-0.05334948,-0.11470193,-0.10683062,0.006505014,-0.06672162,-0.1095178,-0.07157552,-0.05224579,⋯,0.01074269,0.07578441,0.04415781,-0.01997997,-0.0631386,0.00129932,0.04803797,-0.04275848,-0.04492763,-0.03787834
Exhausted.CD8,-0.11621618,-0.10762734,-0.10059042,-0.16015861,-0.113306535,-0.14244392,-0.1693537,-0.09301236,-0.18285099,⋯,-0.17781316,-0.15275585,-0.15469462,-0.13958048,-0.1510342,-0.09902108,-0.12238242,-0.15427864,-0.16620987,-0.20799196


In [13]:
write.table(dan_NES,
            file='../../Data/3/DanaherImmuneCells_TreatmentNaive_NES.txt',
            sep='\t',
            row.names=F,
            col.names=T)

# End script